In [10]:
import sys
import duckdb
import numpy as np
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
np.set_printoptions(threshold=np.inf, linewidth=np.inf, suppress=True)

from data.model.hand import Hand
from data.utils.hand_encoder import HandEncoder

In [11]:
con = duckdb.connect('../data/db/master.db')
id = con.sql('select hand_id from hands where is_showdown = 1 limit 1;').fetchone()[0]
print(id)

3019291764


In [12]:
con.sql(f'select * from actions where hand_id = {id}').show()

┌────────────┬──────────────┬─────────┬─────────────┬────────┬──────────────────┬──────────────┬─────────────┬─────────┬──────────────┐
│  hand_id   │ action_index │  actor  │ action_type │ amount │ total_pot_amount │ street_index │ is_terminal │  cards  │  raw_action  │
│  varchar   │    int64     │ varchar │   varchar   │ double │      double      │     int8     │   boolean   │ varchar │   varchar    │
├────────────┼──────────────┼─────────┼─────────────┼────────┼──────────────────┼──────────────┼─────────────┼─────────┼──────────────┤
│ 3019291764 │            0 │ d       │ dh          │   NULL │             0.75 │            0 │ false       │ p1 ???? │ d dh p1 ???? │
│ 3019291764 │            1 │ d       │ dh          │   NULL │             0.75 │            0 │ false       │ p2 ???? │ d dh p2 ???? │
│ 3019291764 │            2 │ d       │ dh          │   NULL │             0.75 │            0 │ false       │ p3 ???? │ d dh p3 ???? │
│ 3019291764 │            3 │ d       │ dh      

In [13]:
hh = Hand.from_db(id, con)
for a in hh.actions:
    print(f'raw={a.raw_action}, cards={a.cards}')

raw=d dh p1 ????, cards=['p1', '??', '??']
raw=d dh p2 ????, cards=['p2', '??', '??']
raw=d dh p3 ????, cards=['p3', '??', '??']
raw=d dh p4 ????, cards=['p4', '??', '??']
raw=d dh p5 ????, cards=['p5', '??', '??']
raw=d dh p6 ????, cards=['p6', '??', '??']
raw=p3 f, cards=[]
raw=p4 f, cards=[]
raw=p5 cc, cards=[]
raw=p6 f, cards=[]
raw=p1 cc, cards=[]
raw=p2 cc, cards=[]
raw=d db 7hQs2d, cards=['7h', 'Qs', '2d']
raw=p1 cc, cards=[]
raw=p2 cc, cards=[]
raw=p5 cc, cards=[]
raw=d db 8h, cards=['8h']
raw=p1 cc, cards=[]
raw=p2 cc, cards=[]
raw=p5 cc, cards=[]
raw=d db 6c, cards=['6c']
raw=p1 cc, cards=[]
raw=p2 cc, cards=[]
raw=p5 cc, cards=[]
raw=p1 sm 9c4c, cards=['9c', '4c']
raw=p2 sm Jc9d, cards=['Jc', '9d']
raw=p5 sm Ks9s, cards=['Ks', '9s']


In [14]:
con.close()